In [17]:
import numpy as np

In [18]:
# Function to permute bits according to a given order
def permute_des(key, order, length):
    permuted_bits = ''.join(key[order[idx] - 1] for idx in range(length))
    return permuted_bits

# Function to perform left circular shift on a string of bits
def shift(key, shifts):
    key_list = list(key)
    new_key = key_list[shifts:] + key_list[:shifts]
    
    new_key_string = ""
    for c in new_key:
        new_key_string += c
    
    return new_key_string

In [19]:
# Expansion E-box Table
E = [32, 1 , 2 , 3 , 4 , 5 , 4 , 5,
         6 , 7 , 8 , 9 , 8 , 9 , 10, 11,
         12, 13, 12, 13, 14, 15, 16, 17,
         16, 17, 18, 19, 20, 21, 20, 21,
         22, 23, 24, 25, 24, 25, 26, 27,
         28, 29, 28, 29, 30, 31, 32, 1 ]

#Inverse of P
INVP = [9, 17, 23, 31,
	      13, 28,  2, 18,
	      24, 16, 30,  6,
	      26, 20, 10,  1,
	       8, 14, 25,  3,
	       4, 29, 11, 19,
	      32, 12, 22,  7,
	       5, 27, 15, 21
        ]

# S-box Table
sbox = [[[14, 4, 13, 1, 2, 15, 11, 8, 3 , 10, 6, 12, 5, 9, 0, 7],
    [0, 15, 7, 4, 14, 2, 13, 1, 10, 6, 12, 11, 9, 5, 3, 8],
    [4, 1 , 14, 8, 13, 6, 2, 11, 15, 12, 9, 7,3, 10, 5, 0],
    [15, 12, 8,2,4, 9, 1,7 , 5, 11, 3, 14, 10, 0, 6, 13]],

    [[15, 1, 8, 14, 6, 11, 3, 4, 9, 7, 2, 13, 12, 0,5, 10],
    [3, 13, 4, 7, 15, 2, 8, 14, 12, 0, 1, 10, 6, 9, 11, 5],
    [0, 14, 7, 11, 10, 4, 13, 1, 5, 8,12, 6, 9, 3, 2, 15],
    [13, 8, 10, 1, 3, 15, 4, 2,11,6, 7, 12, 0,5, 14, 9]],

    [[10, 0, 9,14,6,3,15,5, 1, 13, 12, 7, 11, 4,2,8],
    [13, 7, 0, 9, 3, 4, 6, 10, 2, 8, 5, 14, 12, 11, 15, 1],
    [13, 6, 4, 9, 8, 15, 3, 0, 11, 1, 2, 12,5, 10, 14, 7],
    [1, 10, 13, 0, 6, 9, 8, 7, 4, 15, 14, 3, 11, 5, 2, 12]],

    [[7, 13, 14, 3, 0, 6, 9, 10, 1, 2, 8, 5, 11, 12, 4, 15],
    [13, 8, 11, 5, 6, 15, 0, 3, 4, 7, 2, 12, 1, 10, 14, 9],
    [10, 6, 9, 0, 12, 11, 7, 13, 15, 1 , 3, 14, 5, 2, 8, 4],
    [3, 15, 0, 6, 10, 1, 13, 8, 9, 4, 5, 11, 12, 7, 2, 14]],

    [[2, 12, 4, 1, 7, 10, 11, 6, 8, 5, 3, 15, 13, 0, 14, 9],
    [14, 11,2, 12, 4, 7, 13, 1, 5, 0, 15, 10, 3, 9, 8, 6],
    [4, 2, 1, 11, 10, 13, 7, 8, 15, 9, 12, 5, 6, 3, 0, 14],
    [11, 8, 12, 7, 1, 14, 2, 13, 6, 15, 0, 9, 10, 4, 5, 3]],

    [[12, 1, 10, 15, 9, 2, 6,8, 0, 13, 3, 4, 14, 7, 5, 11],
    [10, 15, 4, 2, 7, 12, 9, 5, 6, 1, 13, 14, 0, 11, 3, 8],
    [9, 14, 15, 5, 2,8, 12, 3, 7, 0, 4, 10, 1, 13, 11, 6],
    [4, 3, 2, 12, 9, 5, 15, 10, 11, 14, 1, 7, 6, 0, 8, 13]],

    [[4, 11, 2, 14, 15, 0, 8, 13, 3, 12, 9, 7, 5, 10, 6, 1],
    [13, 0, 11, 7, 4, 9, 1, 10, 14, 3, 5, 12, 2, 15, 8, 6],
    [1, 4, 11, 13, 12, 3, 7, 14, 10, 15, 6, 8, 0, 5, 9, 2],
    [6, 11, 13, 8, 1, 4, 10, 7, 9, 5, 0, 15, 14, 2, 3, 12]],

    [[13, 2, 8, 4, 6, 15, 11, 1, 10, 9, 3, 14, 5, 0, 12,7],
    [1, 15, 13, 8, 10, 3, 7, 4, 12, 5, 6, 11, 0, 14, 9, 2],
    [7, 11, 4, 1, 9, 12, 14, 2, 0, 6, 10, 13, 15, 3, 5, 8],
    [2, 1, 14, 7, 4, 10, 8, 13, 15, 12, 9, 0, 3, 5, 6, 11]]]

f2u_mapping={'f': '0000',
             'g': '0001',
             'h': '0010',
             'i': '0011',
             'j': '0100',
             'k': '0101',
             'l': '0110',
             'm': '0111',
             'n': '1000',
             'o': '1001',
             'p': '1010',
             'q': '1011',
             'r': '1100',
             's': '1101',
             't': '1110',
             'u': '1111'}

## Key Schedule
PC2 = [
  14, 17, 11, 24,  1, 5,
  3, 28 ,15,  6, 21, 10,
  23, 19, 12,  4, 26, 8,
  16,  7, 27, 20, 13, 2,
  41, 52, 31, 37, 47, 55,
  30, 40, 51, 45, 33, 48,
  44, 49, 39, 56, 34, 53,
  46, 42, 50, 36, 29, 32
]

## Reverse Final Permutation
RFP = [57, 49, 41, 33, 25, 17, 9,  1,
       59, 51, 43, 35, 27, 19, 11, 3,
       61, 53, 45, 37, 29, 21, 13, 5,
       63, 55, 47, 39, 31, 23, 15, 7,
       58, 50, 42, 34, 26, 18, 10, 2,
       60, 52, 44, 36, 28, 20, 12, 4,
       62, 54, 46, 38, 30, 22, 14, 6,
       64, 56, 48, 40, 32, 24, 16, 8]

#shift table
shift_table = [1, 1, 2, 2,
                2, 2, 2, 2,
                1, 2, 2, 2,
                2, 2, 2, 1 ]

# Key- Compression Table : Compression of key from 56 bits to 48 bits
key_comp = [14, 17, 11, 24, 1, 5,
            3, 28, 15, 6, 21, 10,
            23, 19, 12, 4, 26, 8,
            16, 7, 27, 20, 13, 2,
            41, 52, 31, 37, 47, 55,
            30, 40, 51, 45, 33, 48,
            44, 49, 39, 56, 34, 53,
            46, 42, 50, 36, 29, 32 ]


mapping = {
           'f' : [0,0,0,0],
           'g' : [0,0,0,1],
           'h' : [0,0,1,0],
           'i' : [0,0,1,1],
           'j' : [0,1,0,0],
           'k' : [0,1,0,1],
           'l' : [0,1,1,0],
           'm' : [0,1,1,1],
           'n' : [1,0,0,0],
           'o' : [1,0,0,1],
           'p' : [1,0,1,0],
           'q' : [1,0,1,1],
           'r' : [1,1,0,0],
           's' : [1,1,0,1],
           't' : [1,1,1,0],
           'u' : [1,1,1,1]
          }

# Permutation at start of DES
initial_perm = [58, 50, 42, 34, 26, 18, 10, 2,
                60, 52, 44, 36, 28, 20, 12, 4,
                62, 54, 46, 38, 30, 22, 14, 6,
                64, 56, 48, 40, 32, 24, 16, 8,
                57, 49, 41, 33, 25, 17, 9, 1,
                59, 51, 43, 35, 27, 19, 11, 3,
                61, 53, 45, 37, 29, 21, 13, 5,
                63, 55, 47, 39, 31, 23, 15, 7]

## S Box Permutation Table
sboxper = [ 16,  7, 20, 21,
        29, 12, 28, 17,
         1, 15, 23, 26,
         5, 18, 31, 10,
         2,  8, 24, 14,
        32, 27,  3,  9,
        19, 13, 30,  6,
        22, 11,  4, 25 ]

## Permutation at the end of DES
final_perm = [ 40, 8, 48, 16, 56, 24, 64, 32,
               39, 7, 47, 15, 55, 23, 63, 31,
               38, 6, 46, 14, 54, 22, 62, 30,
               37, 5, 45, 13, 53, 21, 61, 29,
               36, 4, 44, 12, 52, 20, 60, 28,
               35, 3, 43, 11, 51, 19, 59, 27,
               34, 2, 42, 10, 50, 18, 58, 26,
               33, 1, 41, 9, 49, 17, 57, 25 ]

In [22]:
#Computing XORs at IN and OUT of S-boxes for characteristic 40 08 00 00 04 00 00 00

# Read ciphertexts from the file
with open('ciphertexts1.txt', 'r') as file:
    ciphertext_des1 = file.read().split("\n")

hexcipher_des1 = []

for text in ciphertext_des1:
    hex_value = []
    if len(text) == 17 and text[0] != ' ':
        for char in text[0:16]:
            hex_value.extend(mapping[char])
        hexcipher_des1.append(hex_value)

# Inverse the final permutation
invres_des1 = []
for char in hexcipher_des1:
    temp = []
    for i in range(64):
        temp.append(char[RFP[i] - 1])
    invres_des1.append(temp)

# Xor the ciphertext pairs according to differential
resxor_des1 = []
for i in range(0, len(invres_des1) // 2):
    resxor_des1.append([a ^ b for a, b in zip(invres_des1[2*i], invres_des1[2*i+1])])

# Expand Right side block of Round 5
expanded_des1 = []

for block in invres_des1:
    temp = []
    for j in range(48):
        temp.append(block[E[j] - 1])
    expanded_des1.append(temp)

# XOR expanded blocks to compute s-box input
sin_des1 = []
for i in range(0, len(expanded_des1) // 2):
    sin_des1.append([a ^ b for a, b in zip(expanded_des1[2*i], expanded_des1[2*i+1])])

# XORing L5 and R6
L5 = [0, 0, 0, 0, 0, 1] + [0] * 26
inxor = []
for block in resxor_des1:
    inxor.append([a ^ b for a, b in zip(block[32:64], L5)])

# XORing s-box outputs
sxor_des1 = []
for block in inxor:
    temp = []
    for j in range(32):
        temp.append(block[INVP[j] - 1])
    sxor_des1.append(temp)


# Initialize keys array
keys = np.zeros((8, 64))

for i in range(len(sin_des1)):
    if sin_des1[i] == "":
        continue
    for j in range(8):
        inx = int(''.join(map(str, sin_des1[i][j*6:j*6+6])), 2)
        outx = int(''.join(map(str, sxor_des1[i][j*4:j*4+4])), 2)
        inp = int(''.join(map(str, expanded_des1[2*i+1][j*6:j*6+6])), 2)

        for k in range(64):
            a = format(k, '06b')
            b = format(k ^ inx, '06b')
            if outx == sbox[j][int(a[0]) * 2 + int(a[5])][int(a[4]) + 2 * int(a[3]) + int(a[2]) * 4 + int(a[1]) * 8] ^ sbox[j][int(b[0]) * 2 + int(b[5])][int(b[4]) + 2 * int(b[3]) + int(b[2]) * 4 + int(b[1]) * 8]:
                keys[j][k ^ inp] += 1

print(keys)



[[306. 359. 327. 342. 314. 289. 322. 322. 313. 336. 309. 321. 339. 353.
  332. 321. 309. 326. 309. 343. 313. 288. 295. 335. 308. 339. 311. 301.
  334. 330. 329. 313. 310. 322. 331. 312. 333. 323. 298. 313. 322. 340.
  281. 292. 320. 324. 297. 314. 324. 310. 317. 316. 338. 366. 314. 292.
  333. 345. 332. 297. 311. 502. 309. 354.]
 [344. 330. 334. 344. 332. 397. 390. 326. 358. 368. 375. 346. 336. 370.
  362. 354. 327. 348. 332. 381. 336. 356. 405. 363. 362. 345. 351. 367.
  338. 332. 319. 316. 335. 321. 325. 353. 315. 343. 360. 364. 359. 319.
  309. 356. 341. 367. 343. 402. 332. 337. 365. 950. 341. 351. 336. 376.
  341. 344. 379. 346. 348. 349. 337. 326.]
 [333. 308. 308. 322. 316. 335. 324. 288. 303. 308. 329. 320. 322. 308.
  320. 357. 328. 306. 295. 311. 308. 306. 314. 332. 326. 354. 285. 328.
  338. 341. 317. 331. 303. 324. 326. 357. 315. 472. 307. 331. 337. 320.
  312. 343. 304. 340. 329. 339. 334. 306. 329. 313. 308. 299. 305. 316.
  331. 300. 332. 350. 330. 312. 324. 301.]
 [330. 

In [23]:
# Calculate max values, means, and corresponding keys
maxval_des = []
mean_des = []
keyval_des = []

for i in range(len(keys)):
    mean_des.append(int(np.mean(keys[i])))
    current_max = max(keys[i])
    maxval_des.append(current_max)
    keyval_des.append(np.where(keys[i] == current_max)[0][0])

# Print results
print("S-box" + "\t" + "Max" + "\t" + "Mean" + "\t" + "Key")
for i in range(8):
    print("S" + str(i+1) + "\t" + str(maxval_des[i]) + "\t" + str(mean_des[i]) + "\t" + str(keyval_des[i]))


S-box	Max	Mean	Key
S1	502.0	323	61
S2	950.0	358	51
S3	472.0	322	37
S4	434.0	322	7
S5	548.0	331	25
S6	919.0	349	33
S7	627.0	333	21
S8	623.0	332	55


In [26]:
#Computing XORs at IN and OUT of S-boxes for characteristic 40 08 00 00 04 00 00 00


ciphertext_des2 = open('ciphertexts2.txt').read().split("\n")

hexcipher_des2 = []

for text in ciphertext_des2:
    hex_value = []
    if len(text) == 17 and text[0] != ' ':
        for char in text[0: 16]:
            hex_value.extend(mapping[char])
        hexcipher_des2.append(hex_value)

##Inverse the final permutation

invres_des2 = []
for c in hexcipher_des2:
    temp=[]
    for i in range(64):
        temp.extend([c[RFP[i]-1]])
    invres_des2.extend([temp])


##Xoring the ciphertext pairs according to differential

resxor_des2 = []
for i in range(0,len(invres_des2)-1,2):
    resxor_des2.extend([list(np.bitwise_xor(invres_des2[i],invres_des2[i+1]))])


##Expanding Right side block of Round 5

expanded_des2 = []

for i in invres_des2:
    temp = []
    for j in range(48):
        temp.extend([i[E[j]-1]])
    expanded_des2.extend([temp])

##Xoring expanded to compute s box input

sin_des2 = []
for i in range(0,len(expanded_des2)-1,2):
    sin_des2.extend([list(np.bitwise_xor(expanded_des2[i],expanded_des2[i+1]))])

##Xoring L5 and R6

L5 = [0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0]
inxor = []
for i in resxor_des2:
    inxor.extend([list(np.bitwise_xor(i[32:64],L5))])

##Xoring sbox outputs

sxor_des2 =[]
for i in inxor:
    temp = []
    for j in range(32):
        temp.extend([i[INVP[j]-1]])
    sxor_des2.extend([temp])

##Finding the Keys  corresponding to Round 6 for above charcateristic

keys = np.zeros((8,64))

for i in range(len(sin_des2)):
    for j in range(0,8):
        inx = int(''.join([str(s) for s in sin_des2[i][j*6:j*6+6]]),2)
        outx = int(''.join([str(s) for s in sxor_des2[i][j*4:j*4+4]]),2)
        inp = int(''.join([str(s) for s in expanded_des2[2*i+1][j*6:j*6+6]]),2)

        for k in range(0,64):
            a = bin(k)[2:].zfill(6)
            b = bin(k^inx)[2:].zfill(6)
            if outx == sbox[j][int(a[0])*2 + int(a[5])][int(a[4]) + 2 *int(a[3]) + int(a[2]) * 4 + int(a[1])*8]^sbox[j][int(b[0])*2 + int(b[5])][int(b[4]) + 2 *int(b[3]) + int(b[2]) * 4 + int(b[1])*8]:
                keys[j][k^inp]+=1
 
print(keys)


[[ 311.  320.  329.  367.  322.  310.  305.  323.  361.  339.  318.  313.
   285.  322.  322.  336.  307.  323.  343.  303.  345.  341.  308.  335.
   306.  312.  320.  315.  325.  324.  336.  327.  353.  332.  355.  323.
   311.  313.  327.  317.  298.  364.  316.  326.  339.  358.  319.  322.
   293.  319.  304.  345.  319.  304.  308.  324.  304.  306.  315.  346.
   338.  577.  329.  329.]
 [ 313.  321.  319.  298.  298.  309.  348.  326.  325.  349.  326.  344.
   293.  330.  325.  329.  292.  325.  341.  336.  340.  295.  318.  328.
   306.  300.  331.  342.  331.  346.  342.  310.  327.  300.  332.  321.
   321.  307.  346.  331.  312.  345.  319.  332.  311.  297.  311.  312.
   331.  322.  344.  580.  334.  327.  322.  346.  329.  305.  314.  338.
   297.  348.  313.  328.]
 [ 310.  321.  317.  313.  301.  303.  315.  317.  287.  318.  310.  328.
   318.  299.  295.  327.  309.  336.  285.  303.  299.  307.  307.  292.
   279.  328.  343.  299.  329.  306.  316.  299.  322.  2

In [27]:
maxval_des = [] 
mean_des = [] 
keyval_des = [] 
for i in range(len(keys)):     
    mean_des.append(int(keys[i].mean()))     
    current = max(keys[i])     
    maxval_des.append(current)     
    keyval_des.append(np.where(keys[i]==current)[0][0]) 

print("S-box"+ "\t" +"Max" + "\t" + "Mean" + "\t" + "Key") 
for i in range(0,8):     
    print("S"+ str(i+1) +"\t"+ str(maxval_des[i]) + "\t" + str(mean_des[i]) + "\t" + str(keyval_des[i]))

S-box	Max	Mean	Key
S1	577.0	327	61
S2	580.0	327	51
S3	475.0	318	37
S4	1014.0	370	7
S5	631.0	328	25
S6	974.0	353	33
S7	458.0	318	21
S8	422.0	319	55


In [30]:
sb_key = "101101110011XXXXXX000111111100111000011101110110"  
key = ['X'] * 56

# Applying PC2 permutation
for i in range(48):
    key[PC2[i] - 1] = sb_key[i]

# Applying the key schedule
for i in range(6):
    for j in range(shift_table[i]):
        x = key[27]
        y = key[55]

        for k in range(27, 0, -1):
            key[k] = key[k - 1]
            key[k + 28] = key[k + 27]

        key[0] = x
        key[28] = y

print(''.join(key))
miskey = ''.join(key)


X11XX1XX01011X100XX11X11000X0010111X01111000X11X1111X001


In [37]:
poskey_des = []
binlist_des = []

# possible binary values of length 14
for i in range(2**14):
    x = bin(i)[2:]
    binlist_des.append(('0') * (14 - len(x)) + x)

# Generate possible keys by replacing 'X's in the miskey
for i in binlist_des:
    j = 0
    tempkey = list(miskey)
    for k in range(len(tempkey)):
        if tempkey[k] == 'X':
            tempkey[k] = i[j]
            j += 1
    poskey_des.append(''.join(tempkey))

def roundkey_des(initial_key, rounds):
    left_half = initial_key[:28]
    right_half = initial_key[28:]

    round_keys = []

    for i in range(rounds):
        left_half = shift(left_half, shift_table[i])
        right_half = shift(right_half, shift_table[i])

        combined_key = left_half + right_half

        round_keys.append(permute_des(combined_key, key_comp, 48))

    return round_keys

def encryption_des(message, key_schedule, num_rounds):
    message = permute_des(message, initial_perm, 64)

    left_half = message[:32]
    right_half = message[32:]

    for i in range(num_rounds):
        expanded_message = permute_des(right_half, E, 48)
        in_xor = bin(np.bitwise_xor(int(expanded_message, 2), int(key_schedule[i], 2)))[2:].zfill(48)
        
        s_box_output = ''
        for j in range(8):
            row = int(in_xor[j * 6] + in_xor[j * 6 + 5], 2)
            col = int(in_xor[j * 6 + 1:j * 6 + 5], 2)
            s_box_output += bin(sbox[j][row][col])[2:].zfill(4)

        s_box_output = permute_des(s_box_output, sboxper, 32)
        round_xor = bin(np.bitwise_xor(int(left_half, 2), int(s_box_output, 2)))[2:].zfill(32)
        left_half = right_half
        if i != num_rounds - 1:
            right_half = round_xor

    output_message = left_half + right_half
    cipher_text = permute_des(output_message, final_perm, 64)

    return cipher_text

In [38]:
mainkey_des = ''
binplain_des = ''
for char in 'fghijklmnopqrstu':
    binplain_des += f2u_mapping[char]

bincipher = ''
for char in "fmtmsmktqfrhljit":
    bincipher += f2u_mapping[char]

for k in poskey_des:
    key_schedule = roundkey_des(k, 6)
    if encryption_des(binplain_des, key_schedule, 6) == bincipher:
        mainkey_des = k
        print('The key is', k, '\n')
        for i in range(6):
            print('Round', i, 'key is', key_schedule[i])

        break


In [35]:
## seperate key into comma seperated
print(','.join(mainkey_des))

In [36]:
password_des = 'llrrgfrflomstgpfkqnfjrniohirltnm'

for i in range(0, len(password_des), 2):
    binary_chunk = f2u_mapping[password_des[i]] + f2u_mapping[password_des[i+1]]
    decimal_value = int(binary_chunk, 2)
    print(password_des[i:i+2], decimal_value)


ll 102
rr 204
gf 16
rf 192
lo 105
ms 125
tg 225
pf 160
kq 91
nf 128
jr 76
ni 131
oh 146
ir 60
lt 110
nm 135
